In [1]:
# 必要なライブラリをインポート
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

##Tokenizer, 事前学習されたLanguage modelを準備，日本語BERTを指定
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
model = AutoModelForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
##Language modelを評価モードに
model.eval()


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [2]:
##一部がマスクされた文章を用意．マスクは[MASK]で表現
text = 'ある日の暮方の事である。一人の下人が、羅生門の下で[MASK]を待っていた。'
##文章をBERT内で扱われる数値表現に変換
input_ids = tokenizer.encode(text, return_tensors='pt')
##[MASK]の位置を取得しておく．
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]

In [3]:
with torch.no_grad():##計算グラフを作らず勾配計算を行わないようにする．
    results = model(input_ids)##[MASK]部分を推論
    predictions = results[0][0, masked_index].topk(10)# 推論結果上位10件を取得
    pred_ids = predictions.indices.tolist()# 結果をリスト形式に
    
    for i, pred_id in zip(range(len(pred_ids)), pred_ids):##1位から順に表示
        word_ids = [pred_id]
        decoded_word = tokenizer.decode(word_ids)## 数値を実際の単語に変換
        print(str(i+1) + ':' + decoded_word)

1:彼
2:夜
3:帰り
4:旅人
5:[UNK]
6:春
7:人
8:時
9:日
10:彼ら
